<a href="https://colab.research.google.com/github/Ahram-41/Time-Series-Analysis/blob/main/LangChain_Openai_API_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install -q openai
!pip install -q langchain-openai
!pip install -q langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00


In [2]:
# importing openai module into your openai environment
import openai

# assigning API KEY to initialize openai environment
openai.api_key = ''

# in local env
import os
os.environ["OPENAI_API_KEY"] =""

In [11]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [40]:
llm.invoke("how can langsmith help with testing?")

AIMessage(content='Langsmith can help with testing in several ways:\n\n1. Automated testing: Langsmith can be used to write automated test scripts that can be run repeatedly to ensure that the software functions correctly. This can help to identify bugs and issues early in the development process.\n\n2. Test data generation: Langsmith can be used to generate test data that can be used to test the software under different scenarios and conditions. This can help to ensure that the software is robust and can handle a variety of inputs.\n\n3. Test case management: Langsmith can be used to manage test cases and track their execution, making it easier to keep track of which tests have been run and which still need to be completed.\n\n4. Performance testing: Langsmith can be used to simulate high loads and stress on the software to test its performance under different conditions. This can help to identify performance bottlenecks and optimize the software for better performance.\n\nOverall, La

We can also guide its response with a prompt template. Prompt templates convert raw user input to better input to the LLM.

The output of a ChatModel (and therefore, of this chain) is a message. However, it's often much more convenient to work with strings. Let's add a simple output parser to convert the chat message to a string.

In [13]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

combine into a chain:

In [14]:
chain = prompt | llm | output_parser

In [15]:
chain.invoke({"input": "how can langsmith help with testing?"})

"LangSmith is a platform designed to enhance the development, testing, and deployment of AI-driven applications, particularly those that utilize language models. Here are several ways LangSmith can assist with testing:\n\n1. **Automated Test Generation**: LangSmith can help generate automated test cases for your language models. By analyzing the model's expected behavior, it can create diverse test scenarios, including edge cases that might not be immediately apparent.\n\n2. **Test Coverage Analysis**: The platform can assess the coverage of your tests, ensuring that all relevant functionalities of the language model are being evaluated. This helps identify gaps in your testing strategy.\n\n3. **Performance Benchmarking**: LangSmith can facilitate the benchmarking of language models under various conditions. This includes testing for latency, accuracy, and throughput, helping you evaluate how well your model performs in real-world scenarios.\n\n4. **Error Analysis**: With its analytica

# Retriever Chain

 import and use WebBaseLoader

In [19]:
!pip install beautifulsoup4
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 51.7 MB/s eta 0:00:00


In [18]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

Then index it into a vectorstore. This requires a few components, namely an embedding model and a vectorstore.

For embedding models, we once again provide examples for accessing via API or by running local models

In [20]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [21]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [22]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [23]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help with testing by allowing you to visualize test results.'

In [24]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

LangSmith helps with testing in several ways:

1. **Initial Test Set**: Developers can create datasets consisting of inputs and reference outputs to run tests on their LLM applications. These test cases can be uploaded in bulk or generated from application traces, facilitating a more test-driven approach.

2. **Comparison View**: It offers a user-friendly comparison view that allows developers to see results for different application configurations side-by-side, helping to identify regressions in test scores across multiple revisions.

3. **Beta Testing**: During beta testing, LangSmith enables developers to collect data on application performance, understand input types that lead to success or failure, and gather feedback to curate test cases.

4. **Capturing Feedback**: LangSmith allows users to attach feedback scores to logged traces, which helps in identifying problematic responses and focusing on runs that require attention.

5. **Annotating Traces**: Users can send runs to annota

# Agent


In [27]:
from langchain_community.utilities import SerpAPIWrapper

In [30]:
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32009 sha256=0fc83621cbf09e608ea4f6a3c62d1f4dac11e479495aaf75699e1ab3273b0099
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


In [26]:
import os
os.environ["SERPAPI_API_KEY"]=""

In [32]:
from langchain.agents import load_tools
tools = load_tools(["serpapi"])

In [36]:
from langchain.agents import initialize_agent
from langchain.llms import OpenAI

# Initialize an LLM (e.g., OpenAI)
llm = ChatOpenAI(temperature=0)  # Adjust temperature as needed

# Initialize the agent with the tools
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",  # This agent type uses LLM to decide on tool usage
    verbose=True
)

<ipython-input-36-874e123068c4>:8: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent = initialize_agent(


In [38]:
# Use the agent to answer a question
response = agent.run("who is the director of the walking dead series? what is his current age raised to the 0.35 power")
print(response)



> Entering new AgentExecutor chain...
I should use the search tool to find the director of The Walking Dead series and then calculate his current age raised to the 0.35 power.
Action: Search
Action Input: Director of The Walking Dead series
Observation: ['The Walking Dead creator Robert Kirkman is also an executive producer and writer of the television series. On January 20, 2010, AMC officially announced ...', 'Series Directed by (54) · Greg Nicotero · Michael E. Satrazemis · David Boyd · Ernest R. Dickerson · Guy Ferland · Billy Gierhart · Laura Belsey · Jeffrey F. January.', 'The Walking Dead · Creator · Frank Darabont · Stars · Andrew Lincoln · Norman Reedus · Melissa McBride.', 'James joined the cast of Fear the Walking Dead in season four and Amelio joined the cast in season five. ... Greg Nicotero also served as the primary director for ...', 'The Walking Dead season 1 ; Andrew Lincoln · Jon Bernthal · Sarah Wayne Callies · Laurie Holden · Jeffrey DeMunn · Steven Yeun · Chandl